## 7. 함수 데커레이터와 클로저
함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해주는데, 이를 잘 사용하려면 클로저를 먼저 알아야 한다.
클로저는 콜백을 이용한 효율적인 비동기 프로그래밍와 함수형 스타일을 코딩하는 데에 필수적인 기능이다. 


### 7.1. 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

In [0]:
@decorate
def target():
  print("running target()")
  
# 위의 구문은 아래와 동일하게 작동한다
def target():
  print("running target()")

target = decorate(target)

In [2]:
def deco(func):
  def inner():
    print("running inner()")
  return inner

@deco
def target():
  print("running target()")
  
print(target())
print(target) # <function deco.<locals>.inner at 0x7f1e399ea730> => deco 함수 local에 선언된 inner라는 뜻?

running inner()
None
<function deco.<locals>.inner at 0x7f1e399ea730>


### 7.2. 파이썬이 데커레이터를 실행하는 시점

데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬이 모듈을 로딩하는 시점, import time에 실행된다. 

In [4]:
# registration.py

registry = []

def register(func):
  print("running register(%s)" % func)
  registry.append(func)
  return func

@register
def f1():
  print("running f1()")
  
@register
def f2():
  print("running f2()")
  
  
def f3():
  print("running f3()")
  
def main():
  print("running main()")
  print("registry ->", registry)
  f1() # 명시적으로 호출될 때만 실행된다.
  f2()
  f3()
  
  
if __name__ == '__main__':
  main()
  
'''
running register(<function f1 at 0x7f1e399d9ea0>)  # import 시점에 이미 한번 실행
running register(<function f2 at 0x7f1e39a247b8>)
running main()
registry -> [<function f1 at 0x7f1e399d9ea0>, <function f2 at 0x7f1e39a247b8>]
running f1()
running f2()
running f3()
'''

import registration # 위의 모듈을 임포트하면 바로 register() 실행

running register(<function f1 at 0x7f1e399d9ea0>)
running register(<function f2 at 0x7f1e39a247b8>)
running main()
registry -> [<function f1 at 0x7f1e399d9ea0>, <function f2 at 0x7f1e39a247b8>]
running f1()
running f2()
running f3()


- 일반적으로 데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈은 분리해서 구현한다.
- register() 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환한다. 실제 코드에서 대부분의 데커레이터는 내부 함수를 정의해서 반환한다.

### 7.3. 데커레이터로 개선한 전략 패턴

데커레이터를 사용하면 6장의 전자상거래 프로모션 할인 코드를 개선할 수 있다. best_promo 함수에서 promos 리스트에 함수명을 반복해서 사용하는데, 새로운 할인 전략이 추가되었을 때 여기에 누락되면 버그가 발생할 수 있다.

In [0]:
promos = [] # 리스트에 promotion 함수들이 임포트 시점에 append

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity(order): 
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
    
@promotion    
def bulk_item(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1

    return discount
    
@promotion    
def large_order(order): 
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0
  
def best_promo(order):
  return max(promo(order) for promo in promos)

### 7.4. 변수 범위 규칙


In [0]:
def f1(a):
  print(a)
  print(b) 

f1(3) # b가 선언되어 있지 않기 때문에 에러
  
b = 6
f1(3) # 정상 동작

In [7]:
b=7

def f2(a):
  print(a)
  print(b)
  b = 9
  
f2(3) #local variable 'b' referenced before assignment

3


UnboundLocalError: ignored

파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역변수로 판단한다. 파이썬은 변수가 선언되어 있기를 요구하지 않지만, 함수 본체 안에서 할당된 변수는 지역 변수로 판단한다. 함수 안에 할당하는 문장이 있지만 인터프리터가 b를 전역 변수로 다루기 원한다면, global 키워드를 사용해야한다.

In [11]:
b=7

def f2(a):
  global b
  print(a)
  print(b)
  b = 9 # 전역의 b를 9로 바꿈
  print(b) 
  print()
  
f2(3)
print(b)
f2(3)
b=30
print(b)

3
7
9

9
3
9
9

30


### 7.5. 클로저

클로저는 함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수이다. 클로저를 사용하면 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다.

In [13]:
class Averager() :
  def __init__(self):
    self.series = []
    
  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total / len(self.series)
  
  
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [14]:
def make_averager():
  # 지역함수이기 때문에 리턴하면 없어질텐데?
  series = [] # => 자유변수 : 지역 범위에 바인딩되어 있지 않은 변수
  
  def average(new_value):
    series.append(new_value)
    total = sum(series)
    return total / len(series)
  
  return averager


avg = make_averager() # 내부 함수 avearge
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [18]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)
print(avg.__closure__) 

# series에 대한 바인딩은 avg() 함수의 __closure__ 속성에 저장된다
print(avg.__closure__[0].cell_contents)

('new_value', 'total')
('series',)
(<cell at 0x7f1e39a45a68: list object at 0x7f1e399e4ec8>,)
[10, 11, 12]


### 7.6. nonlocal 선언

In [0]:
def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    count += 1
    total += new_value
    return total/count
  
  return averager

avg = make_averager()
avg(10) # UnboundLocalError: local variable 'count' referenced before assignment

위의 예제에서 averager() 함수에서 count, total에 할당 연산이 일어나기 때문에 파이썬은 이들을 함수 내의 지역변수라고 인식한다. 앞선 예제에서 series 변수에 할당을 하지 않았기 때문에 이 문제가 생기지 않았다. 

그렇다면 숫자, 문자열, 튜플 등 불변형은 읽을 수만 있고 값을 갱신할 수 없는 것인가? 너무 사용성이 떨어지는데? 이 문제를 해결하기 위해서 파이썬 3에서 nonlocal이라는 선언이 추가되었다. 변수를 nonlocal로 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유 변수임을 나타낸다.

In [20]:
def make_averager():
  count = 0
  total = 0
  
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total/count
  
  return averager

avg = make_averager()
avg(10) # UnboundLocalError: local variable 'count' referenced before assignment

10.0

### 7.7 간단한 데커레이터 구현하기


In [0]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter()
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  
  return clocked

In [26]:
# clockdeco_demo.py

@clock
def snooze(seconds):
  time.sleep(seconds)
  
@clock
def factorial(n):
  return 1 if n <2 else n*factorial(n-1)

if __name__ == "__main__":
  print("*" * 40, 'Calling snooze(.123)')
  snooze(.123)
  print("*" * 40, 'Calling factorial(6)')
  print("6! = ", factorial(6))

**************************************** Calling snooze(.123)
[5977.26167964s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[5977.26233413s] factorial(1) -> 1
[5977.26238277s] factorial(2) -> 2
[5977.26248188s] factorial(3) -> 6
[5977.26252821s] factorial(4) -> 24
[5977.26268006s] factorial(5) -> 120
[5977.26277453s] factorial(6) -> 720
6! =  720


In [28]:
factorial.__name__ # 인터프리터가 clock 함수를 거쳐 factorial에 clocked를 할당

'clocked'

In [0]:
# 위의 구현은 키워드 인수를 받지 못하고,
# 기존의 __name__과 __doc__ 속성을 가려버리는 문제가 있다.

import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time()
    name = func.__name__
    arg_list = []
    if args:
      arg_list.append(", ".join(repr(arg) for arg in args))
    if kwargs:
      pairs = ["%s=%s" % (k, w) for k, w in sorted(kwargs.items())]
      arg_list.append(", ".join(pairs))
      
    arg_str = ', '.join(arg_list)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked
      
    

### 7.8. 표준 라이브러리에서 제공하는 데커레이터

#### 7.8.1. functools.lru_cache()를 이용한 메모이제이션
메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다. LRU(Least Recently Used) 즉, 오랫동안 사용하지 않은 항목을 버림으로써 캐시가 무한정 커지지 않게 해준다.

In [31]:
@clock
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n-2) + fibonacci(n-1)

if __name__ == "__main__":
  print(fibonacci(6)) # fibonacci(1), fibonacci(2)가 불필요하게 많이 호출됨

[1533527806.72174120s] fibonacci(0) -> 0
[1533527806.72635269s] fibonacci(1) -> 1
[1533527806.72674203s] fibonacci(2) -> 1
[1533527806.72694492s] fibonacci(1) -> 1
[1533527806.72719836s] fibonacci(0) -> 0
[1533527806.72734833s] fibonacci(1) -> 1
[1533527806.72748566s] fibonacci(2) -> 1
[1533527806.72762823s] fibonacci(3) -> 2
[1533527806.72776341s] fibonacci(4) -> 3
[1533527806.72793698s] fibonacci(1) -> 1
[1533527806.72807503s] fibonacci(0) -> 0
[1533527806.72811627s] fibonacci(1) -> 1
[1533527806.72825551s] fibonacci(2) -> 1
[1533527806.72830558s] fibonacci(3) -> 2
[1533527806.72886586s] fibonacci(0) -> 0
[1533527806.72900820s] fibonacci(1) -> 1
[1533527806.72905993s] fibonacci(2) -> 1
[1533527806.72916770s] fibonacci(1) -> 1
[1533527806.72930431s] fibonacci(0) -> 0
[1533527806.72935534s] fibonacci(1) -> 1
[1533527806.72945404s] fibonacci(2) -> 1
[1533527806.72950196s] fibonacci(3) -> 2
[1533527806.72959733s] fibonacci(4) -> 3
[1533527806.72972655s] fibonacci(5) -> 5
[1533527806.7297

In [32]:
@functools.lru_cache() # 데커레이터를 일반함수처럼 호출해야 한다
def fibonacci(n):
  if n < 2:
    return n
  return fibonacci(n-2) + fibonacci(n-1)

if __name__ == "__main__":
  print(fibonacci(6)) # fibonacci(1), fibonacci(2)가 불필요하게 많이 호출됨

[1533527895.21098161s] fibonacci(0) -> 0
[1533527895.21234250s] fibonacci(1) -> 1
[1533527895.21249151s] fibonacci(2) -> 1
[1533527895.21263933s] fibonacci(3) -> 2
[1533527895.21544814s] fibonacci(4) -> 3
[1533527895.21624732s] fibonacci(5) -> 5
[1533527895.21635771s] fibonacci(6) -> 8
8


`functools.lru_cache(maxsize=128, typed=False)` 가 전체 함수 모양 => 두개의 선택적 인수를 이용해서 설정할 수 있다. 
maxsize는 얼마나 많은 호출을 저장할지 결정한다. typed가 True로 설정된 경우 인수의 자료형이 다르면 결과를 따로 저장한다.

#### 7.8.2. 단일 디스패치를 이용한 범용 함수


In [0]:
import html
def htmlize(obj):
  content = html.escape(repr(obj))
  return '<pre>{}</pre>'.format(content)

In [35]:
# 자료형 타입에 따라서 다르게 인코딩하고 싶어
# but 파이썬에서는 메서드나 함수의 오버로딩을 지원하지 않으므로 자료형별로 다른 htmlize를 생성할 수 없다
# if/else로 처리하기에도 코드가 너무 커지기 때문에 비효율적이다

print(htmlize({1,2,3}))
print(htmlize(abs))
print(htmlize("Heimlich & Co.\n- a game"))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co.\n- a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [0]:
from functools import singledispatch
from collections import abc
import numbers
import html

# 객체형을 다룰 기반 함수
@singledispatch 
def htmlize(obj): 
  content = html.escape(repr(obj))
  return '<pre>{}</pre>'.format(content)

# 기반함수.register(객체형)
@htmlize.register(str)
def _(text):
  content = html.escape(text).replace('\n', "<br>\n")
  return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
  return '<pre>{0} (0x{0:x})</pre>'.format(n)

# 같은 함수로 여러 자료형을 처리하고 싶을 때
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
  inner = '<li>\n</li>'.join(htmlize(item) for item in seq)
  return "<ul>\n<li>"+inner+"</li>\n</ul>"

  

In [38]:
print(htmlize({1,2,3}))
print(htmlize(abs))
print(htmlize("Heimlich & Co.\n- a game"))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co.<br>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p><li>
</li><pre>66 (0x42)</pre><li>
</li><pre>{1, 2, 3}</pre></li>
</ul>


### 7.9. 누적된 데커레이터

하나의 함수 f()에 두 데커레이터 @d1과 @d2를 차례대로 적용하면, `f = d1(d2(f))`와 동일하다.


### 7.10. 매개변수화된 데커레이터
다른 인수를 받는 데커레이터를 만들 수 있을까?

#### 7.10.1. 매개변수화된 등록 데커레이터
register 함수에 activate 인수를 만들도록 해보자. 새로 만든 register는 개념적으로 데커레이터가 아니라 데커레이터 팩토리이다. 호출되면 대상 함수에 적용할 실제 데커레이터를 반환하기 때문이다.

In [42]:
registry = set()

def register(activate=True):
  def decorate(func):
    print("running register(activate=%s)->decorate(%s)" % (activate, func))
    if activate:
      registry.add(func)
    else :
      registry.discard(func)
      
    return func
  return decorate

@register(activate=False)
def f1():
  print("running f1()")
  
@register()
def f2():
  print("running f2()")
  
def f3():
  print("running f3()")
  
print(registry)

running register(activate=False)->decorate(<function f1 at 0x7f1e39a0de18>)
running register(activate=True)->decorate(<function f2 at 0x7f1e39a242f0>)
{<function f2 at 0x7f1e39a242f0>}


In [43]:
print(register()(f3))
print(registry)
print(register(activate=False)(f2))
print(registry)


running register(activate=True)->decorate(<function f3 at 0x7f1e399eae18>)
<function f3 at 0x7f1e399eae18>
{<function f3 at 0x7f1e399eae18>, <function f2 at 0x7f1e39a242f0>}
running register(activate=False)->decorate(<function f2 at 0x7f1e39a242f0>)
<function f2 at 0x7f1e39a242f0>
{<function f3 at 0x7f1e399eae18>}


#### 7.10.2. 매개변수화된 clock 데커레이터


In [47]:
import time
DEFAULT_FMT = '[{elapsed: 0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
  def decorate(func):
    def clocked(*args):
      t0 = time.time()
      _result = func(*args)
      elapsed = time.time()
      name = func.__name__
      arg_str = ', '.join(repr(arg) for arg in args)
      result = repr(_result)
      print(fmt.format(**locals()))
      return _result
    return clocked
  return decorate


if __name__ == "__main__":
  @clock()
  def snooze(seconds):
    time.sleep(seconds)
    
  for i in range(3):
    snooze(.123)

[ 1533530966.02932167s] snooze((0.123,)) -> None
[ 1533530966.15277147s] snooze((0.123,)) -> None
[ 1533530966.27618623s] snooze((0.123,)) -> None


In [48]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
  time.sleep(seconds)
  
for i in range(3):
  snooze(.123)

snooze: 1533531121.2064767s
snooze: 1533531121.3299048s
snooze: 1533531121.4533782s


In [49]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
  time.sleep(seconds)
  
for i in range(3):
  snooze(.123)

snooze((0.123,)) dt=1533531173.182s
snooze((0.123,)) dt=1533531173.306s
snooze((0.123,)) dt=1533531173.429s
